<a href="https://colab.research.google.com/github/sambit9238/DataScience/blob/master/fraud_invention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import itertools
%matplotlib inline

In [2]:
!wget https://snap.stanford.edu/data/soc-sign-bitcoinotc.csv.gz
!gunzip -k soc-sign-bitcoinotc.csv.gz

--2019-05-27 08:55:34--  https://snap.stanford.edu/data/soc-sign-bitcoinotc.csv.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 397258 (388K) [application/x-gzip]
Saving to: ‘soc-sign-bitcoinotc.csv.gz’

soc-sign-bitcoinotc 100%[===================>] 387.95K  2.09MB/s    in 0.2s    

2019-05-27 08:55:35 (2.09 MB/s) - ‘soc-sign-bitcoinotc.csv.gz’ saved [397258/397258]



In [3]:
!wget https://snap.stanford.edu/data/soc-sign-bitcoinalpha.csv.gz
!gunzip -k soc-sign-bitcoinalpha.csv.gz

--2019-05-27 08:55:40--  https://snap.stanford.edu/data/soc-sign-bitcoinalpha.csv.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149480 (146K) [application/x-gzip]
Saving to: ‘soc-sign-bitcoinalpha.csv.gz’

soc-sign-bitcoinalp 100%[===================>] 145.98K  --.-KB/s    in 0.1s    

2019-05-27 08:55:41 (1.19 MB/s) - ‘soc-sign-bitcoinalpha.csv.gz’ saved [149480/149480]



In [4]:
!ls

sample_data		      soc-sign-bitcoinotc.csv
soc-sign-bitcoinalpha.csv     soc-sign-bitcoinotc.csv.gz
soc-sign-bitcoinalpha.csv.gz


In [5]:
!wget https://github.com/Jhird/TrustKeeper/raw/master/1_ORIGINAL_NETWORK.csv
!wget https://github.com/Jhird/TrustKeeper/raw/master/2_TRIMMED_NETWORK.csv
#!wget https://github.com/Jhird/TrustKeeper/blob/master/3_TEST_NETWORK.csv

--2019-05-27 08:55:52--  https://github.com/Jhird/TrustKeeper/raw/master/1_ORIGINAL_NETWORK.csv
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Jhird/TrustKeeper/master/1_ORIGINAL_NETWORK.csv [following]
--2019-05-27 08:55:52--  https://raw.githubusercontent.com/Jhird/TrustKeeper/master/1_ORIGINAL_NETWORK.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1973980 (1.9M) [text/plain]
Saving to: ‘1_ORIGINAL_NETWORK.csv’

1_ORIGINAL_NETWORK. 100%[===================>]   1.88M  --.-KB/s    in 0.05s   

2019-05-27 08:55:52 (36.9 MB/s) - ‘1_ORIGINAL_NETWORK.csv’ saved [1973980/1973980]

--2019-05-27 0

In [6]:
!ls

1_ORIGINAL_NETWORK.csv	   soc-sign-bitcoinalpha.csv.gz
2_TRIMMED_NETWORK.csv	   soc-sign-bitcoinotc.csv
sample_data		   soc-sign-bitcoinotc.csv.gz
soc-sign-bitcoinalpha.csv


In [0]:
#Preprocessing the transaction data

In [8]:
trim_network = pd.read_csv('2_TRIMMED_NETWORK.csv')
trim_network.head()

,SOURCE,TARGET,RATING,TIMESTAMP,EPISODE
0,35,3952,1,1.365623e+09,21958
1,2125,5379,-1,1.392045e+09,32086
2,5227,5854,1,1.417038e+09,35751
3,6013,13492,-1,1.393304e+09,32395
4,6013,13495,-1,1.392008e+09,32075


In [9]:
trim_network.shape

(49822, 5)

In [0]:
#extract node features
def GetNodeFeatures(GDF,n):
    if n==0:
        return ['in_degree','pos_in_edges','neg_in_edges','out_degree','pos_out_edges','neg_out_edges']
    # Get sub-dataframes
    outgoing = GDF[GDF['SOURCE']==n].copy().reset_index(drop=True)
    incoming = GDF[GDF['TARGET']==n].copy().reset_index(drop=True)
    
    # Compute node degree (incoming)
    in_degree = len(incoming)
    
    # Compute node degree (outgoing)
    out_degree = len(outgoing)
    
    # Compute number of incoming nodes with positive ratings
    pos_in_edges = len([i for i in incoming.RATING.values if i>0])
    
    # Computer number of incoming nodes with negative ratings
    neg_in_edges = len([i for i in incoming.RATING.values if i<0])
    
    # Repeat for outgoing edges
    pos_out_edges = len([i for i in outgoing.RATING.values if i>0])
    
    # Computer number of outgoing nodes with negative ratings
    neg_out_edges = len([i for i in outgoing.RATING.values if i<0])
    
    return np.array([in_degree,pos_in_edges,neg_in_edges,out_degree,pos_out_edges,neg_out_edges])

In [0]:
# Define list of nodes and sort it in ascending order
nodes = list( set(trim_network['SOURCE']) | set(trim_network['TARGET']) )
nodes.sort()

In [12]:
len(nodes)

9664

In [0]:
# The node feature matrix creation
NF = np.zeros([len(nodes),6])
for i,n in enumerate(nodes):
    NF[i,:] = np.array(GetNodeFeatures(trim_network,n))

In [0]:
# Create a dataframe containing all these node features
NF_df = pd.DataFrame(columns=GetNodeFeatures(trim_network,0),data=NF,index=nodes)
NF_df.to_csv('trimmed_node_features.csv')

In [0]:
NF_df = pd.read_csv('trimmed_node_features.csv',index_col=0)

In [16]:
NF_df.head()

,in_degree,pos_in_edges,neg_in_edges,out_degree,pos_out_edges,neg_out_edges
1,198.0,198.0,0.0,184.0,177.0,7.0
2,36.0,35.0,1.0,36.0,34.0,2.0
3,20.0,11.0,9.0,0.0,0.0,0.0
4,40.0,40.0,0.0,52.0,49.0,3.0
5,3.0,3.0,0.0,2.0,2.0,0.0


In [0]:
f = open('trimmed_network.edgelist','w')
for i in range(len(trim_network)):
    s = str(trim_network['SOURCE'][i])
    t = str(trim_network['TARGET'][i])
    f.write(s+' '+t+'\n')
f.close()

In [18]:
len(trim_network)

49822

In [19]:
!git clone https://github.com/aditya-grover/node2vec.git

Cloning into 'node2vec'...
remote: Enumerating objects: 108, done.
remote: Total 108 (delta 0), reused 0 (delta 0), pack-reused 108
Receiving objects: 100% (108/108), 51.58 KiB | 10.32 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [20]:
!ls

1_ORIGINAL_NETWORK.csv	soc-sign-bitcoinalpha.csv     trimmed_network.edgelist
2_TRIMMED_NETWORK.csv	soc-sign-bitcoinalpha.csv.gz  trimmed_node_features.csv
node2vec		soc-sign-bitcoinotc.csv
sample_data		soc-sign-bitcoinotc.csv.gz


In [21]:
!python ./node2vec/src/main.py --input ./trimmed_network.edgelist  \
                               --output ./trimmed_network.emb \
                               --dimensions 14 --num-walks 25 --iter 15

Walk iteration:
1 / 25
2 / 25
3 / 25
4 / 25
5 / 25
6 / 25
7 / 25
8 / 25
9 / 25
10 / 25
11 / 25
12 / 25
13 / 25
14 / 25
15 / 25
16 / 25
17 / 25
18 / 25
19 / 25
20 / 25
21 / 25
22 / 25
23 / 25
24 / 25
25 / 25
y
Traceback (most recent call last):
  File "./node2vec/src/main.py", line 104, in <module>
    main(args)
  File "./node2vec/src/main.py", line 100, in main
    learn_embeddings(walks)
  File "./node2vec/src/main.py", line 88, in learn_embeddings
    model.save_word2vec_format(args.output)
  File "/usr/local/lib/python2.7/dist-packages/gensim/models/word2vec.py", line 1289, in save_word2vec_format
    raise DeprecationWarning("Deprecated. Use model.wv.save_word2vec_format instead.")


In [22]:
!ls

1_ORIGINAL_NETWORK.csv	soc-sign-bitcoinalpha.csv     trimmed_network.edgelist
2_TRIMMED_NETWORK.csv	soc-sign-bitcoinalpha.csv.gz  trimmed_node_features.csv
node2vec		soc-sign-bitcoinotc.csv
sample_data		soc-sign-bitcoinotc.csv.gz


In [0]:
#format dictionary like node : embedding
n = []
e = []
with open('./trimmed_network.emb') as fin:
    for line in fin:
        node_emb = line.strip().split()
        n.append(node_emb[0])
        e.append(node_emb[1:])

In [0]:
n = n[1:]
n = [int(i) for i in n]
embs = np.zeros([len(e)-1,14])
for i in range(1,len(e)):
    embs[i-1] = e[i]
embs.shape